# 链家租房数据爬虫与分析

## 1. 环境配置与依赖安装

In [1]:
# 安装依赖
# !pip install catboost plotly beautifulsoup4 pandas numpy matplotlib scikit-learn fake-useragent requests seaborn

## 2. 配置文件 (Settings)

In [2]:
# 链家：https://xm.lianjia.com/zufang/
# 原址：https://github.com/Dyasxue/BUPT_python_final_exp_lianjiaSpider

from pathlib import Path

# 项目目录配置
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / "data"
IMAGE_DIR = PROJECT_ROOT / "images"

DATA_DIR.mkdir(parents=True, exist_ok=True)
IMAGE_DIR.mkdir(parents=True, exist_ok=True)

# 重要：请替换为你自己在链家网登录后的 Cookie
COOKIE = """lianjia_uuid=21879120-1b21-49b8-ae51-bf462eeea672; crosSdkDT2019DeviceId=-n8sihi-e5a4d0-yh0lk4hhy11ahyr-49vxcihat; ftkrc_=775a8b9e-29c8-4701-9d81-299c3b5bf38c; lfrc_=0794bd25-2752-46e5-999f-f277ab4340ff; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2219b4b9589c6d51-049b59bc23a5d88-4c657b58-2073600-19b4b9589c715e0%22%2C%22%24device_id%22%3A%2219b4b9589c6d51-049b59bc23a5d88-4c657b58-2073600-19b4b9589c715e0%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E4%BB%98%E8%B4%B9%E5%B9%BF%E5%91%8A%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22https%3A%2F%2Fwww.baidu.com%2Fother.php%22%2C%22%24latest_referrer_host%22%3A%22www.baidu.com%22%2C%22%24latest_search_keyword%22%3A%22%E9%93%BE%E5%AE%B6%22%2C%22%24latest_utm_source%22%3A%22baidu%22%2C%22%24latest_utm_medium%22%3A%22pinzhuan%22%2C%22%24latest_utm_campaign%22%3A%22wyfs%22%2C%22%24latest_utm_content%22%3A%22biaotimiaoxau%22%2C%22%24latest_utm_term%22%3A%22biaoti%22%7D%7D; select_city=350200; GUARANTEE_POPUP_SHOW=true; GUARANTEE_BANNER_SHOW=true; login_ucid=2000000512872949; lianjia_token=2.00146b5cf64f63e0ef05c675c7c01d6fe3; lianjia_token_secure=2.00146b5cf64f63e0ef05c675c7c01d6fe3; security_ticket=Km1prrtLg0Rg6VBikAI5CXZQJqbJqniEkzfZBIAFsRUKMwzf3ygXT4qIzHEY+m01fEyaWujvPlNDuIIDgRrUWuVJXsRKMYvgVz/tDNPdaYKQ/s6UC9VSeLRjuRnHL9TD2PsfzmQTbaJ77eOSZHdMY01opdG0VPzeFbw10olMk9Y=; lianjia_ssid=e07cfb39-a97c-47bc-a4c7-75311a9b540d; hip=DlrVud7_QcpqSJqmQ1pb9ZK_3mRILkADZ4p7nfsgd_ldNVKpWSNYljrzceIA75Hc6CsFSLFri3HcMvtqKAAcaLrZMsMtAox8c5KYH9fLGvaupn7XDDzy58bbmVb6gAlAn5XLoqufE9apWDqjDYRxlxp8e9GkP14Vd_v0iB-33gopZTwC53PUOJ8-94hvc7xSN6TZGfVg_SThBoKscBNh8n3qk97HF7QDsPKLXnoN07omTUQ_meyk-VXcFlcyT1ZZfNSeiIEA2X9BxK_Ovn_R_MAKYbFvDmQUhCXBGg%3D%3D; srcid=eyJ0Ijoie1wiZGF0YVwiOlwiYTZjZDRkMzVmZjE3NjcxOTg5YTFhMzE4NzQ4NGMzZDBmNDMzYzYwNWFlNGM4MjVkMmJhOTdlNmM3NWI3MWUyMjFmZmJkZDdlNTNkNWQzOGQ3MjA0ZmE5MjEwOTA3OGUwNWQ5NzVkMDYyNTJmYjgwYmNiYWQxMGMzNDZmZmFkZDA3OWU0N2JlMGVmMjRmOGI1Mjc4YmIzYWNhYzY2OTNiZGUxMmUwMzFjODBjMTY1M2Q5YTdjOTRiNDMwMGVhYmJjMmJjZjU5ZjBmM2JlMzRkZGMwYTFmNzgzZmJlYjI2MTg2NzhkMDUwZjFmMTIzNjhiMWEyMDE2MDM3NWI1ZmRkYVwiLFwia2V5X2lkXCI6XCIxXCIsXCJzaWduXCI6XCIzYTU3Mjg1ZlwifSIsInIiOiJodHRwczovL3htLmxpYW5qaWEuY29tL3p1ZmFuZy8iLCJvcyI6IndlYiIsInYiOiIwLjEifQ=="""

# 城市映射
CITIES_MAP = {
    'bj': '北京',
    'sh': '上海',
    'gz': '广州',
    'sz': '深圳',
    'wh': '武汉',
    'cd': '成都',
    'hz': '杭州',
    'xm': '厦门'
}

# 爬虫参数
START_PAGE = 1
END_PAGE = 50
MIN_DELAY = 6
MAX_DELAY = 8
MAX_FAILURES = 3

# 分析目标城市
ANALYSIS_CITY = '厦门'

## 3. 工具类 (Utilities)

### 3.1 CSV 工具类

In [3]:
from typing import Union, List, Optional
import pandas as pd


class CsvUtil:
    @staticmethod
    def save_data(
            data: Union[pd.DataFrame, List[dict]],
            prefix: str,
            city_name: Optional[str] = None
    ) -> Path:
        target_dir = DATA_DIR

        df = pd.DataFrame(data) if not isinstance(data, pd.DataFrame) else data

        filename = f"{prefix}_{city_name}.csv" if city_name else f"{prefix}.csv"
        file_path = target_dir / filename

        df.to_csv(file_path, index=False, encoding='utf-8-sig')
        return file_path

    @staticmethod
    def load_data(
            prefix: str,
            merge_pattern: bool = False
    ) -> Optional[pd.DataFrame]:
        target_dir = DATA_DIR

        if merge_pattern:
            files = list(target_dir.glob(f"{prefix}_*.csv"))
        else:
            files = list(target_dir.glob(f"{prefix}.csv"))

        if not files:
            return None

        try:
            df_list = []
            for f in files:
                df_temp = pd.read_csv(f, encoding='utf-8-sig')
                df_list.append(df_temp)

            if not df_list:
                return None

            return pd.concat(df_list, ignore_index=True)
        except Exception as e:
            return None

### 3.2 图表保存工具类

In [4]:
import logging
import plotly.graph_objects as go

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(message)s',
    datefmt='%H:%M:%S'
)

logger = logging.getLogger(__name__)


class ImageUtil:
    @staticmethod
    def save_plot(
            fig: go.Figure,
            filename: str,
            fmt: str = 'png',
            scale: float = 2.0
    ) -> str:
        output_dir = IMAGE_DIR

        file_path = output_dir / f"{filename}.{fmt}"

        try:
            if fmt == 'html':
                fig.write_html(str(file_path))
            else:
                fig.write_image(str(file_path), scale=scale)

            return str(file_path)
        except Exception as e:
            return ""

## 4. 阶段一：数据爬取

In [5]:
import random
import re
import time
from typing import Dict, List, Any

import requests
from bs4 import BeautifulSoup, Tag
from fake_useragent import UserAgent


class LianJiaSpider:
    _PATTERNS = {
        'layout': re.compile(r'(\d+)室(\d+)厅(\d+)卫'),
        'area': re.compile(r'(\d+\.?\d*)㎡'),
        'orientation': re.compile(r'/\s*([\u4e00-\u9fa5\s]+)\s*/'),
        'floor_level': re.compile(r'([\u4e00-\u9fa5]+)楼层'),
        'total_floors': re.compile(r'（(\d+)层）')
    }

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': UserAgent().random,
            'Cookie': COOKIE
        })

    def run(self, specific_cities):
        for city_code, city_name in specific_cities.items():
            data = self._crawl_city(city_name, city_code)
            CsvUtil.save_data(data, prefix='raw', city_name=city_name)

    def _crawl_city(self, city_name: str, city_code: str) -> List[Dict]:
        logger.info(f">>> 启动抓取: {city_name} ({city_code})")
        items = []
        consecutive_failures = 0

        for page in range(START_PAGE, END_PAGE + 1):
            logger.info(f"[{city_name}] 正在处理第 {page} 页...")

            page_data = self._fetch_page(city_code, page, city_name)

            if page_data:
                items.extend(page_data)
                consecutive_failures = 0
                logger.info(f"    √ 获取 {len(page_data)} 条")
            else:
                consecutive_failures += 1
                logger.info(f"    × 第 {page} 页无数据")

            if consecutive_failures >= MAX_FAILURES:
                logger.error(f"连续失败触发熔断: {city_name}")
                break

            time.sleep(random.uniform(MIN_DELAY, MAX_DELAY))

        return items

    def _fetch_page(self, city_code: str, page: int, city_name: str) -> List[Dict]:
        url = f"https://{city_code}.lianjia.com/zufang/pg{page}/"

        try:
            resp = self.session.get(url, timeout=10)
            resp.raise_for_status()

            soup = BeautifulSoup(resp.content, 'html.parser')
            cards = soup.select('div.content__list--item')

            results = []
            for card in cards:
                results.append(self._parse_card(card, city_name))
            return results
        except Exception as e:
            logger.error(f"请求异常 (Page {page}): {e}")
            return []

    def _parse_card(self, card: Tag, city_name: str) -> Dict[str, Any]:
        def _text(selector: str) -> str:
            el = card.select_one(selector)
            return el.get_text(strip=True) if el else ""

        def _regex(text: str, key: str, dtype: type = str, default: Any = None) -> Any:
            match = self._PATTERNS[key].search(text)
            if match:
                try:
                    return dtype(match.group(1)) if match.lastindex == 1 else match.groups()
                except (ValueError, IndexError):
                    pass
            return default

        desc = _text('p.content__list--item--des')
        floor_txt = _text('p.content__list--item--des span.hide')
        title = _text('p.content__list--item--title')

        locs = []
        for a in card.select('p.content__list--item--des a')[:3]:
            locs.append(a.get_text(strip=True))
        for _ in range(3 - len(locs)):
            locs.append('')

        layout = self._PATTERNS['layout'].search(desc)
        rooms = [0, 0, 0]
        if layout:
            rooms = []
            for x in layout.groups():
                rooms.append(int(x))

        price_str = _text('span.content__list--item-price em')

        tag_texts = []
        for i in card.select('p.content__list--item--bottom i'):
            tag_texts.append(i.get_text(strip=True))
        tags = '|'.join(tag_texts)

        return {
            'city': city_name,
            'title': title,
            'rent_type': '整租' if '整租' in title else ('合租' if '合租' in title else '独栋'),
            'district': locs[0],
            'sub_district': locs[1],
            'community': locs[2],
            'area_sqm': _regex(desc, 'area', float, 0.0),
            'bedrooms': rooms[0],
            'living_rooms': rooms[1],
            'bathrooms': rooms[2],
            'orientation': _regex(desc, 'orientation', str, '').strip(),
            'floor_level': _regex(floor_txt, 'floor_level', str, ''),
            'total_floors': _regex(floor_txt, 'total_floors', int, 0),
            'tags': tags,
            'platform': _text('p.content__list--item--brand span.brand'),
            'update_time': _text('p.content__list--item--brand span.content__list--item--time'),
            'price_rmb': price_str
        }

## 5. 阶段二：数据清洗

In [6]:
from datetime import datetime, timedelta


class DataCleaner:
    COLS_TEXT = [
        'city', 'title', 'rent_type', 'district', 'sub_district',
        'community', 'orientation', 'floor_level', 'tags',
        'platform', 'update_time', 'price_rmb'
    ]
    COLS_NUM = ['area_sqm', 'bedrooms', 'living_rooms', 'bathrooms', 'total_floors']

    _PATTERN_PRICE = re.compile(r'(\d+\.?\d*)')
    _PATTERN_REL_DATE = re.compile(r'(\d+)\s*(天|周|个月|月|年)前')
    _DATE_OFFSET_MAP = {'天': 1, '周': 7, '月': 30, '年': 365}

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.now = datetime.now()

    @classmethod
    def execute_task(cls) -> pd.DataFrame:
        raw_df = CsvUtil.load_data("raw", merge_pattern=True)

        if raw_df is None or raw_df.empty:
            logger.warning("跳过清洗：无原始数据")
            return pd.DataFrame()

        cleaner = cls(raw_df)
        cleaned_df = cleaner.process()

        CsvUtil.save_data(cleaned_df, prefix='cleaned')
        return cleaned_df

    def process(self) -> pd.DataFrame:
        initial_len = len(self.df)
        logger.info(f"开始清洗，原始数据量: {initial_len}")

        self.df = (
            self.df
            .pipe(self._clean_text)
            .pipe(self._clean_numeric)
            .pipe(self._parse_prices)
            .pipe(self._standardize_dates)
            .pipe(self._remove_outliers)
        )

        dropped = initial_len - len(self.df)
        logger.info(f"清洗完成: {initial_len} -> {len(self.df)} (剔除 {dropped} 条)")
        return self.df

    def _clean_text(self, df: pd.DataFrame) -> pd.DataFrame:
        cols = df.columns.intersection(self.COLS_TEXT)

        df[cols] = df[cols].fillna('未知').astype(str).apply(lambda x: x.str.strip())
        df[cols] = df[cols].replace(r'^(nan|NaN|None|NULL|)$', '未知', regex=True)

        return df

    def _clean_numeric(self, df: pd.DataFrame) -> pd.DataFrame:
        cols = df.columns.intersection(self.COLS_NUM)

        for col in cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

        return df

    def _parse_prices(self, df: pd.DataFrame) -> pd.DataFrame:
        def _calc_stats(val: str) -> pd.Series:
            raw_nums = self._PATTERN_PRICE.findall(str(val))
            nums = []

            for x in raw_nums:
                nums.append(float(x))

            if not nums:
                return pd.Series([0.0, 0.0, 0.0])

            return pd.Series([min(nums), max(nums), sum(nums) / len(nums)])

        stats = df['price_rmb'].apply(_calc_stats)
        stats.columns = ['price_min', 'price_max', 'price_avg']

        return pd.concat([df, stats], axis=1)

    def _standardize_dates(self, df: pd.DataFrame) -> pd.DataFrame:
        def _parse(date_str: str) -> str:
            if '今天' in date_str:
                return self.now.strftime('%Y-%m-%d')

            match = self._PATTERN_REL_DATE.search(date_str)

            if match:
                val, unit = match.groups()
                unit_key = unit.replace('个', '')
                days = int(val) * self._DATE_OFFSET_MAP.get(unit_key, 0)
                return (self.now - timedelta(days=days)).strftime('%Y-%m-%d')

            return date_str

        df['clean_date'] = df['update_time'].apply(_parse)

        return df

    def _remove_outliers(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df[df['price_avg'] > 0]

        if df.empty:
            return df

        low = df['price_avg'].quantile(0.03)
        high = df['price_avg'].quantile(0.97)

        return df[(df['price_avg'] >= low) & (df['price_avg'] <= high)]

## 6. 阶段三：数据可视化

In [7]:
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots


class RentalDataVisualizer:
    LABELS_MAP = {
        'city': '城市',
        'price_avg': '月租均价 (元)',
        'price_per_sqm': '每平米单价 (元/㎡)',
        'is_subway': '房源类型',
        'district': '行政区',
        'count': '房源数量',
        'range': '价格区间'
    }

    COLOR_SEQ = px.colors.qualitative.Plotly

    def __init__(self, df: pd.DataFrame):
        self.df = df[df['district'] != '未知'].copy()

        self.df['price_per_sqm'] = self.df['price_avg'] / self.df['area_sqm']

        self.df['is_subway'] = np.where(
            self.df['tags'].str.contains('近地铁'), '地铁房', '普通房'
        )

    def run(self):
        logger.info("开始生成数据可视化图表...")

        self._plot_city_comparison()
        self._plot_subway_premium()

        target_city = ANALYSIS_CITY
        city_data = self.df[self.df['city'] == target_city]

        if city_data.empty:
            logger.warning(f"未找到城市 {target_city} 的数据，跳过该城市深度分析图表")
            return

        self._plot_district_price_distribution(city_data, target_city)
        self._plot_district_top10(city_data, target_city)
        self._plot_price_segments(city_data, target_city)

        logger.info("所有图表已保存完成。")

    def _apply_style(self, fig, title: str):
        fig.update_layout(
            title={
                'text': title,
                'x': 0.5,
                'y': 0.95,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            template="plotly_white",
            font=dict(family="Microsoft YaHei", size=12),
            margin=dict(l=40, r=40, t=80, b=40),
            showlegend=True
        )
        return fig

    def _plot_city_comparison(self):
        metrics = (
            self.df
            .groupby('city')[['price_avg', 'price_per_sqm']]
            .mean()
            .reset_index()
        )

        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.add_trace(
            go.Bar(
                x=metrics['city'],
                y=metrics['price_avg'],
                name="月租均价",
                marker_color=self.COLOR_SEQ[0],
            ),
            secondary_y=False
        )

        fig.add_trace(
            go.Scatter(
                x=metrics['city'],
                y=metrics['price_per_sqm'],
                name="每平米单价",
                mode='lines+markers',
                line=dict(color=self.COLOR_SEQ[1], width=3)
            ),
            secondary_y=True
        )

        fig.update_yaxes(title_text="月租均价 (元)", secondary_y=False)
        fig.update_yaxes(title_text="每平米单价 (元/㎡)", secondary_y=True, showgrid=False)
        fig.update_xaxes(title_text="城市")

        self._apply_style(fig, "各城市租金水平对比")

        ImageUtil.save_plot(fig, "各城市租金水平对比")

    def _plot_subway_premium(self):
        metrics = (
            self.df
            .groupby('is_subway')['price_avg']
            .mean()
            .reset_index()
        )

        fig = px.bar(
            metrics,
            x='is_subway',
            y='price_avg',
            labels=self.LABELS_MAP,
            color='is_subway',
            color_discrete_sequence=self.COLOR_SEQ,
            text_auto='.2f'
        )

        fig.update_traces(textposition='outside')

        self._apply_style(fig, "地铁房 vs 普通房：月租均价对比")

        ImageUtil.save_plot(fig, "地铁房 vs 普通房：月租均价对比")

    def _plot_district_price_distribution(self, df: pd.DataFrame, city: str):
        fig_box = px.box(
            df,
            x='district',
            y='price_avg',
            labels=self.LABELS_MAP,
            color='district',
            color_discrete_sequence=self.COLOR_SEQ
        )

        fig_box.update_xaxes(categoryorder='median ascending')

        self._apply_style(fig_box, f"{city} - 各区域租金分布")

        ImageUtil.save_plot(fig_box, f"{city} - 各区域租金分布")

    def _plot_district_top10(self, df: pd.DataFrame, city: str):
        top10 = (
            df
            .groupby('district')['price_avg']
            .mean()
            .nlargest(10)
            .reset_index()
        )

        fig_bar = px.bar(
            top10,
            x='price_avg',
            y='district',
            orientation='h',
            labels=self.LABELS_MAP,
            color='price_avg',
            color_continuous_scale='Blues'
        )

        fig_bar.update_yaxes(autorange="reversed")

        self._apply_style(fig_bar, f"{city} - 租金最贵行政区 Top 10")

        ImageUtil.save_plot(fig_bar, f"{city} - 租金最贵行政区 Top 10")

    def _plot_price_segments(self, df: pd.DataFrame, city: str):
        bins = [0, 2000, 4000, 6000, 8000, 10000, float('inf')]
        labels = ['2k以下', '2k-4k', '4k-6k', '6k-8k', '8k-1w', '1w以上']

        counts = pd.cut(df['price_avg'], bins=bins, labels=labels).value_counts().reset_index()
        counts.columns = ['range', 'count']

        fig = px.pie(
            counts,
            values='count',
            names='range',
            labels=self.LABELS_MAP,
            color_discrete_sequence=self.COLOR_SEQ,
            hole=0.4
        )

        fig.update_traces(textposition='inside', textinfo='percent+label')

        self._apply_style(fig, f"{city} - 房源价格区间占比")

        ImageUtil.save_plot(fig, f"{city} - 房源价格区间占比")

## 7. 阶段四：价格预测建模

In [8]:
from typing import Tuple

from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder


class ModelPipeline:
    CATEGORICAL_FEATURES = ['city', 'rent_type', 'district', 'sub_district', 'orientation', 'floor_level']
    NUMERIC_FEATURES = ['area_sqm', 'bedrooms', 'living_rooms', 'bathrooms', 'total_floors', 'is_subway']
    TARGET_COLUMN = 'price_avg'

    FEATURE_DISPLAY_MAP = {
        'city': '城市', 'rent_type': '租赁方式', 'district': '行政区',
        'sub_district': '商圈', 'orientation': '朝向', 'floor_level': '楼层等级',
        'area_sqm': '面积(㎡)', 'bedrooms': '室数', 'living_rooms': '厅数',
        'bathrooms': '卫数', 'total_floors': '总楼层', 'is_subway': '是否近地铁'
    }

    COLOR_SEQ = px.colors.qualitative.Plotly

    def __init__(self, raw_df: pd.DataFrame):
        self.all_feature_names = None
        self.raw_df = raw_df
        self.performance_metrics: List[Dict] = []

    def run(self):
        features, target = self._prepare_data()
        self.all_feature_names = features.columns.tolist()

        train_features, test_features, train_target, test_target = train_test_split(
            features, target, test_size=0.2, random_state=42
        )

        logger.info("正在训练 CatBoost 模型...")
        catboost_model = self._train_catboost(train_features, train_target, test_features, test_target)
        self._evaluate(catboost_model, test_features, test_target, "CatBoost")

        logger.info("正在训练 RandomForest 模型...")
        rf_model, test_features_encoded = self._train_random_forest(train_features, train_target, test_features)
        self._evaluate(rf_model, test_features_encoded, test_target, "RandomForest")

        self._plot_metrics_comparison()
        self._plot_importance(catboost_model)

    def _prepare_data(self) -> Tuple[pd.DataFrame, pd.Series]:
        df = self.raw_df.copy()
        df['is_subway'] = df['tags'].str.contains('近地铁', na=False).astype(int)
        return df[self.CATEGORICAL_FEATURES + self.NUMERIC_FEATURES], df[self.TARGET_COLUMN]

    def _train_catboost(self, train_x, train_y, test_x, test_y):
        train_pool = Pool(train_x, train_y, cat_features=self.CATEGORICAL_FEATURES)
        test_pool = Pool(test_x, test_y, cat_features=self.CATEGORICAL_FEATURES)

        model = CatBoostRegressor(
            iterations=1000,
            learning_rate=0.05,
            depth=5,
            loss_function='RMSE',
            verbose=0,
            early_stopping_rounds=50,
            allow_writing_files=False
        )
        model.fit(train_pool, eval_set=test_pool, use_best_model=True)
        return model

    def _train_random_forest(self, train_x, train_y, test_x):
        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        train_x_encoded = train_x.copy()
        test_x_encoded = test_x.copy()

        train_x_encoded[self.CATEGORICAL_FEATURES] = encoder.fit_transform(train_x[self.CATEGORICAL_FEATURES])
        test_x_encoded[self.CATEGORICAL_FEATURES] = encoder.transform(test_x[self.CATEGORICAL_FEATURES])

        model = RandomForestRegressor(
            n_estimators=1000,
            max_depth=20,
            n_jobs=-1,
            random_state=42
        )
        model.fit(train_x_encoded, train_y)
        return model, test_x_encoded

    def _evaluate(self, model, features, target, model_name: str):
        target_pred = model.predict(features)
        
        rmse = np.sqrt(mean_squared_error(target, target_pred))
        mae = mean_absolute_error(target, target_pred)
        r2 = r2_score(target, target_pred)

        logger.info(f"模型评估: {model_name}: RMSE={rmse:.3f}, MAE={mae:.3f}, R2={r2:.3f}")

        self.performance_metrics.extend([
            {'Model': model_name, 'Metric': 'RMSE', 'Value': rmse},
            {'Model': model_name, 'Metric': 'MAE', 'Value': mae},
            {'Model': model_name, 'Metric': 'R2', 'Value': r2}
        ])

    def _apply_style(self, fig, title: str):
        fig.update_layout(
            title={'text': title, 'x': 0.5, 'y': 0.95, 'xanchor': 'center', 'yanchor': 'top'},
            template="plotly_white",
            font=dict(family="Microsoft YaHei", size=12),
            margin=dict(l=40, r=40, t=80, b=40),
            showlegend=True
        )
        return fig

    def _plot_metrics_comparison(self):
        metrics_df = pd.DataFrame(self.performance_metrics)
        fig = make_subplots(
            rows=1,
            cols=2,
            subplot_titles=("模型拟合度 (R2)", "误差指标 (RMSE/MAE)")
        )

        r2_df = metrics_df[metrics_df['Metric'] == 'R2']
        fig.add_trace(
            go.Bar(
                x=r2_df['Model'],
                y=r2_df['Value'],
                name='R2 Score',
                text=r2_df['Value'].round(4),
                textposition='auto',
                marker_color=self.COLOR_SEQ[0]
            ),
            row=1,
            col=1
        )

        rmse_df = metrics_df[metrics_df['Metric'] == 'RMSE']
        fig.add_trace(
            go.Bar(
                x=rmse_df['Model'],
                y=rmse_df['Value'],
                name='RMSE',
                marker_color=self.COLOR_SEQ[1]
            ),
            row=1,
            col=2
        )

        mae_df = metrics_df[metrics_df['Metric'] == 'MAE']
        fig.add_trace(
            go.Bar(
                x=mae_df['Model'],
                y=mae_df['Value'],
                name='MAE',
                marker_color=self.COLOR_SEQ[2]
            ),
            row=1,
            col=2
        )

        fig.update_layout(barmode='group')

        self._apply_style(fig, "模型性能评估对比")

        ImageUtil.save_plot(
            fig=fig,
            filename="模型性能对比图"
        )

    def _plot_importance(self, model):
        importance_df = pd.DataFrame({
            'feature': self.all_feature_names,
            'score': model.get_feature_importance()
        })
        importance_df['feature_cn'] = importance_df['feature'].map(self.FEATURE_DISPLAY_MAP)
        importance_df = importance_df.sort_values('score', ascending=True).tail(10)

        fig = px.bar(
            importance_df,
            x='score',
            y='feature_cn',
            orientation='h',
            labels={
                'score': '贡献度指数',
                'feature_cn': '特征名称'
            },
            color='score',
            color_continuous_scale='Viridis'
        )

        fig.update_layout(coloraxis_showscale=False)

        self._apply_style(fig, "CatBoost 核心特征贡献排行")

        ImageUtil.save_plot(
            fig=fig,
            filename="CatBoost 核心特征贡献排行"
        )

## 8. 主流程执行

In [9]:
def check_missing_cities() -> dict:
    existing_files = list(DATA_DIR.glob("raw_*.csv"))

    existing_names = set()
    for p in existing_files:
        parts = p.stem.split('_')
        existing_names.add(parts[1])

    missing_cities = {}
    for code, name in CITIES_MAP.items():
        if name not in existing_names:
            missing_cities[code] = name

    return missing_cities

### 8.1 执行阶段一：数据爬取

In [10]:
missing_cities = check_missing_cities()

if missing_cities:
    logger.info(f">>> 阶段 1/4: 开始爬取 (目标: {list(missing_cities.values())})")
    LianJiaSpider().run(missing_cities)
else:
    logger.info(">>> 阶段 1/4: 数据已完整，跳过爬取")

[21:38:10] >>> 阶段 1/4: 数据已完整，跳过爬取


### 8.2 执行阶段二：数据清洗

In [11]:
logger.info(">>> 阶段 2/4: 数据清洗")
df_clean = DataCleaner.execute_task()

# 查看清洗后的数据
if not df_clean.empty:
    display(df_clean.head(10))
    print(f"\n清洗后数据量: {len(df_clean)} 条")

[21:38:10] >>> 阶段 2/4: 数据清洗
[21:38:10] 开始清洗，原始数据量: 7680
[21:38:11] 清洗完成: 7680 -> 7218 (剔除 462 条)


,city,title,rent_type,district,sub_district,community,area_sqm,bedrooms,living_rooms,bathrooms,...,floor_level,total_floors,tags,platform,update_time,price_rmb,price_min,price_max,price_avg,clean_date
0,上海,整租·杨家浜小区 1室1厅 南,整租,杨浦,控江路,杨家浜小区,36.00,1,1,1,...,低,3,自营|新上|近地铁|精装|押一付一|随时看房|首次出租,贝壳优选,今天维护,3200,3200.0,3200.0,3200.0,2026-01-07
1,上海,整租·金杨六街坊 1室1厅 南,整租,浦东,金杨,金杨六街坊,41.07,1,1,1,...,中,6,官方核验|近地铁|随时看房,链家,1天前维护,3400,3400.0,3400.0,3400.0,2026-01-06
2,上海,整租·东方明珠大宁公寓 2室2厅 东南/南,整租,静安,大宁,东方明珠大宁公寓,95.18,2,2,2,...,低,25,自营|新上|押一付一|双卫生间|随时看房|首次出租,贝壳优选,今天维护,7500,7500.0,7500.0,7500.0,2026-01-07
3,上海,独栋·方隅服务公寓 上海会展旗舰店 【康复医院陪护】近华山西院康复医院 可做饭 陪护优选拎包...,独栋,未知,未知,未知,50.00,1,1,1,...,未知,0,独栋公寓|月租|精装|开放厨房|押一付一,方隅服务公寓,1天前维护,4670,4670.0,4670.0,4670.0,2026-01-06
4,上海,整租·绿地海域笙晖(公寓) 3室1厅 南,整租,宝山,杨行,绿地海域笙晖(公寓),96.32,3,1,1,...,低,16,自营|新上|精装|押一付一|随时看房|首次出租,贝壳优选,今天维护,6100,6100.0,6100.0,6100.0,2026-01-07
5,上海,独栋·保利和寓 上海保利和寓光合社区 央企保租房·首开特惠 15号线元江路地铁口 接驳车 1室1厅,独栋,未知,未知,未知,34.02,1,1,1,...,未知,0,独栋公寓|拎包入住|近地铁|精装|有阳台|开放厨房|押一付一,保利和寓,22天前维护,2330,2330.0,2330.0,2330.0,2025-12-16
6,上海,整租·首创禧悦(二期) 3室2厅 南,整租,青浦,盈浦,首创禧悦(二期),113.94,3,2,2,...,中,8,官方核验|自营|新上|近地铁|精装|押一付一|双卫生间|随时看房|首次出租,贝壳优选,今天维护,5000,5000.0,5000.0,5000.0,2026-01-07
7,上海,整租·古银杏苑 2室2厅 东南,整租,浦东,三林,古银杏苑,92.12,2,2,1,...,中,18,官方核验|近地铁|随时看房,链家,未知,6000,6000.0,6000.0,6000.0,未知
8,上海,整租·宝钢三村 2室1厅 南,整租,宝山,淞宝,宝钢三村,46.85,2,1,1,...,高,6,自营|新上|押一付一|随时看房|首次出租,贝壳优选,今天维护,2500,2500.0,2500.0,2500.0,2026-01-07
9,上海,独栋·保利和寓 上海保利和寓光合社区 近颛桥万达！央企保租房 民水民电 24H安保 首租南向...,独栋,未知,未知,未知,34.02,1,1,1,...,未知,0,独栋公寓|拎包入住|近地铁|精装|有阳台|开放厨房|押一付一,保利和寓,22天前维护,2418,2418.0,2418.0,2418.0,2025-12-16



清洗后数据量: 7218 条


### 8.3 执行阶段三：数据可视化

In [16]:
if not df_clean.empty:
    logger.info(">>> 阶段 3/4: 数据可视化")
    RentalDataVisualizer(df_clean).run()
else:
    logger.warning("跳过可视化：无清洗后数据")

[21:31:39] >>> 阶段 3/4: 数据可视化
[21:31:39] 开始生成数据可视化图表...
[21:31:40] 所有图表已保存完成。


### 8.4 执行阶段四：价格预测建模

In [13]:
if not df_clean.empty:
    logger.info(">>> 阶段 4/4: 价格预测建模")
    ModelPipeline(df_clean).run()
else:
    logger.warning("跳过建模：无清洗后数据")

[21:31:40] >>> 阶段 4/4: 价格预测建模
[21:31:40] 正在训练 CatBoost 模型...
[21:33:02] 模型评估: CatBoost: RMSE=1614.604, MAE=985.772, R2=0.804
[21:33:02] 正在训练 RandomForest 模型...
[21:33:07] 模型评估: RandomForest: RMSE=1637.441, MAE=971.654, R2=0.798
[21:39:18] All navigates done, putting them all in queue.
[21:39:18] Getting tab from queue (has 1)
[21:39:18] Got 3887
[21:39:18] Processing CatBoost_.png
[21:39:18] Sending big command for CatBoost_.png.
[21:39:18] Sent big command for CatBoost_.png.
[21:39:18] Reloading tab 3887 before return.
[21:39:18] Putting tab 3887 back (queue size: 0).
[21:39:18] Waiting for all cleanups to finish.
[21:39:18] Exiting Kaleido
[21:39:18] Closing browser.
[21:39:20] Closing browser.
[21:39:20] TemporaryDirectory.cleanup() worked.
[21:39:20] shutil.rmtree worked.
[21:39:20] Cancelling tasks.
[21:39:20] Exiting Kaleido/Choreo
[21:39:20] TemporaryDirectory.cleanup() worked.
[21:39:20] shutil.rmtree worked.
[21:39:20] TemporaryDirectory.cleanup() worked.
[21:39:20] shutil.rmt

## 9. 完成

所有阶段已执行完毕！

- 爬取的原始数据保存在 `data/raw_*.csv`
- 清洗后的数据保存在 `data/cleaned.csv`
- 可视化图表保存在 `images/` 目录下